<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/99_GenerarVisualizacionesPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Generador de PDFs Consolidados de Visualizaciones
Trabajo Fin de Máster - Evaluación Comparativa de Técnicas de Segmentación

Autor: Iesus
Tutor: Miguel Alejandro Ponce Proaño
Universidad Oberta de Catalunya (UOC)
Fecha: Noviembre 2025

Descripción:
-----------
Genera documentos PDF consolidados agrupando todas las visualizaciones
de segmentación por modelo y configuración.

Estructura de salida:
    - Un PDF por cada combinación modelo+configuración
    - Cada PDF contiene las 20 fotografías procesadas
    - Diseño profesional con portada, metadatos y layout optimizado

Ejemplo de uso:
    python 05_generador_pdfs_visualizaciones.py --directorio-base /resultados --directorio-salida /pdfs
"""

In [2]:
import argparse
import json
import logging
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass
from datetime import datetime

import numpy as np
from PIL import Image
from reportlab.lib.pagesizes import A4, landscape
from reportlab.lib.units import cm
from reportlab.pdfgen import canvas
from reportlab.lib.colors import HexColor
from reportlab.lib.utils import ImageReader

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'reportlab'

In [ ]:
@dataclass
class ConfiguracionModelo:
    """Configuración de un modelo de segmentación."""
    codigo_config: str
    modelo_base: str
    variante: str
    configuracion: str
    ruta_visualizaciones: Path
    visualizaciones: List[Path]

    @property
    def nombre_modelo(self) -> str:
        """Nombre legible del modelo."""
        mapeo = {
            'mask2former': 'Mask2Former',
            'oneformer': 'OneFormer',
            'sam': 'SAM 2.0',
            'yolo': 'YOLOv8-seg',
            'bodypix': 'BodyPix'
        }
        return mapeo.get(self.modelo_base.lower(), self.modelo_base)

    @property
    def nombre_configuracion(self) -> str:
        """Nombre legible de la configuración."""
        # Extraer información legible del código de configuración
        partes = self.codigo_config.split('_')

        # Intentar identificar dataset y umbral
        dataset = ''
        umbral = ''
        sensibilidad = ''

        if 'coco' in self.codigo_config.lower():
            dataset = 'COCO-instance'
        elif 'ade' in self.codigo_config.lower():
            dataset = 'ADE20K-semantic'

        # Buscar patrón de umbral T0_X
        for i, parte in enumerate(partes):
            if parte.startswith('T') and i > 0:
                # Extraer valor del umbral
                umbral_valor = parte[1:].replace('_', '.')
                umbral = f"umbral {umbral_valor}"

        # Buscar sensibilidad
        if 'alta' in self.codigo_config:
            sensibilidad = 'alta sensibilidad'
        elif 'baja' in self.codigo_config:
            sensibilidad = 'baja sensibilidad'
        elif 'maxima' in self.codigo_config or 'máxima' in self.codigo_config:
            sensibilidad = 'máxima sensibilidad'
        elif 'media' in self.codigo_config:
            sensibilidad = 'media sensibilidad'
        elif 'ultra' in self.codigo_config:
            sensibilidad = 'ultra sensible'

        # Construir nombre descriptivo
        elementos = [e for e in [dataset, sensibilidad, umbral] if e]
        return ' - '.join(elementos) if elementos else self.configuracion

In [ ]:
class EscaneadorVisualizaciones:
    """Escanea directorios para encontrar visualizaciones por configuración."""

    def __init__(self, directorio_base: Path):
        """
        Args:
            directorio_base: Directorio raíz con estructura de modelos
        """
        self.directorio_base = Path(directorio_base)

        if not self.directorio_base.exists():
            raise FileNotFoundError(f"Directorio base no existe: {self.directorio_base}")

    def escanear(self) -> List[ConfiguracionModelo]:
        """
        Escanea recursivamente el directorio base buscando carpetas 'visualizaciones'.

        Estructura esperada (flexible):
            modelo/variante/.../configuracion/visualizaciones/*.png

        Returns:
            Lista de configuraciones encontradas
        """
        configuraciones = []

        logger.info(f"Escaneando recursivamente: {self.directorio_base}")

        # Buscar TODOS los directorios llamados 'visualizaciones' de forma recursiva
        for dir_viz in self.directorio_base.rglob('visualizaciones'):
            if not dir_viz.is_dir():
                continue

            # Buscar archivos PNG en este directorio
            imagenes = sorted(list(dir_viz.glob('*.png')))

            if not imagenes:
                logger.debug(f"Directorio vacío: {dir_viz}")
                continue

            # Extraer la ruta relativa desde el directorio base
            ruta_relativa = dir_viz.relative_to(self.directorio_base)
            partes_ruta = list(ruta_relativa.parts)

            # La última parte siempre es 'visualizaciones', quitarla
            partes_ruta = partes_ruta[:-1]

            if not partes_ruta:
                logger.warning(f"Ruta inesperada (sin partes): {dir_viz}")
                continue

            # Construir código de configuración: todas las partes unidas con '_'
            codigo_config = '_'.join(partes_ruta)

            # Extraer modelo base (primera parte de la ruta)
            modelo_base = partes_ruta[0]

            # Variante (segunda parte si existe)
            variante = partes_ruta[1] if len(partes_ruta) > 1 else ''

            # Configuración (todo excepto modelo base)
            configuracion_nombre = '_'.join(partes_ruta[1:]) if len(partes_ruta) > 1 else ''

            config = ConfiguracionModelo(
                codigo_config=codigo_config,
                modelo_base=modelo_base,
                variante=variante,
                configuracion=configuracion_nombre,
                ruta_visualizaciones=dir_viz,
                visualizaciones=imagenes
            )

            configuraciones.append(config)
            logger.info(
                f"✓ Encontrado: {config.codigo_config} "
                f"({len(imagenes)} imágenes) - {config.nombre_configuracion}"
            )

        if not configuraciones:
            logger.warning("No se encontraron directorios 'visualizaciones' con imágenes PNG")
        else:
            logger.info(f"\nTotal configuraciones encontradas: {len(configuraciones)}")

        return sorted(configuraciones, key=lambda x: (x.modelo_base, x.codigo_config))

In [ ]:
class GeneradorPDF:
    """Genera PDFs consolidados con visualizaciones."""

    # Configuración de diseño
    COLOR_PRIMARIO = HexColor('#1e3a8a')  # Azul académico
    COLOR_SECUNDARIO = HexColor('#64748b')  # Gris
    COLOR_ACENTO = HexColor('#0ea5e9')  # Azul claro

    def __init__(self, directorio_salida: Path):
        """
        Args:
            directorio_salida: Directorio donde guardar los PDFs
        """
        self.directorio_salida = Path(directorio_salida)
        self.directorio_salida.mkdir(parents=True, exist_ok=True)

    def generar_pdf(self, config: ConfiguracionModelo) -> Path:
        """
        Genera un PDF para una configuración.

        Args:
            config: Configuración del modelo

        Returns:
            Ruta al PDF generado
        """
        # Nombre del archivo PDF
        nombre_pdf = f"{config.codigo_config}.pdf"
        ruta_pdf = self.directorio_salida / nombre_pdf

        logger.info(f"Generando PDF: {nombre_pdf}")

        # Crear canvas
        c = canvas.Canvas(str(ruta_pdf), pagesize=A4)

        # Generar portada
        self._generar_portada(c, config)

        # Generar páginas de visualizaciones
        for idx, imagen_path in enumerate(config.visualizaciones, 1):
            self._generar_pagina_visualizacion(c, imagen_path, idx, len(config.visualizaciones))

        # Generar página resumen (grid)
        self._generar_pagina_resumen(c, config)

        # Guardar PDF
        c.save()

        logger.info(f"✓ PDF generado: {ruta_pdf} ({len(config.visualizaciones) + 2} páginas)")

        return ruta_pdf

    def _generar_portada(self, c: canvas.Canvas, config: ConfiguracionModelo):
        """Genera la portada del PDF."""
        width, height = A4

        # Banner superior
        c.setFillColor(self.COLOR_PRIMARIO)
        c.rect(0, height - 4*cm, width, 4*cm, fill=True, stroke=False)

        # Título principal
        c.setFillColor(HexColor('#ffffff'))
        c.setFont('Helvetica-Bold', 24)
        titulo_y = height - 2*cm
        c.drawCentredString(width/2, titulo_y, "Evaluación de Segmentación")

        # Subtítulo
        c.setFont('Helvetica', 16)
        c.drawCentredString(width/2, titulo_y - 0.8*cm, "Fotografía de Retrato")

        # Información del modelo
        info_y = height - 6*cm
        c.setFillColor(self.COLOR_PRIMARIO)
        c.setFont('Helvetica-Bold', 18)
        c.drawCentredString(width/2, info_y, config.nombre_modelo)

        # Configuración
        c.setFillColor(self.COLOR_SECUNDARIO)
        c.setFont('Helvetica', 14)
        c.drawCentredString(width/2, info_y - 0.8*cm, config.nombre_configuracion)

        # Código técnico
        c.setFont('Courier', 10)
        c.drawCentredString(width/2, info_y - 1.6*cm, config.codigo_config)

        # Metadatos
        meta_y = height - 10*cm
        c.setFillColor(self.COLOR_SECUNDARIO)
        c.setFont('Helvetica', 11)

        metadatos = [
            f"Total de visualizaciones: {len(config.visualizaciones)}",
            f"Fecha de generación: {datetime.now().strftime('%d/%m/%Y %H:%M')}",
            f"Directorio fuente: {config.ruta_visualizaciones.name}"
        ]

        for i, meta in enumerate(metadatos):
            c.drawCentredString(width/2, meta_y - i*0.6*cm, meta)

        # Pie de página
        c.setFont('Helvetica', 9)
        c.setFillColor(self.COLOR_SECUNDARIO)
        pie_texto = "Trabajo Fin de Máster - UOC | Autor: Iesus | Tutor: Miguel Alejandro Ponce Proaño"
        c.drawCentredString(width/2, 2*cm, pie_texto)

        c.showPage()

    def _generar_pagina_visualizacion(
        self,
        c: canvas.Canvas,
        imagen_path: Path,
        numero: int,
        total: int
    ):
        """
        Genera una página con una visualización.

        Args:
            c: Canvas de ReportLab
            imagen_path: Ruta a la imagen
            numero: Número de imagen actual
            total: Total de imágenes
        """
        width, height = A4

        # Encabezado
        c.setFillColor(self.COLOR_PRIMARIO)
        c.setFont('Helvetica-Bold', 12)
        c.drawString(2*cm, height - 1.5*cm, f"Visualización {numero}/{total}")

        c.setFont('Helvetica', 10)
        c.setFillColor(self.COLOR_SECUNDARIO)
        c.drawString(2*cm, height - 2*cm, imagen_path.stem)

        # Cargar y escalar imagen
        try:
            img = Image.open(imagen_path)
            img_reader = ImageReader(img)

            # Área disponible para la imagen (dejando márgenes)
            area_ancho = width - 4*cm
            area_alto = height - 5*cm

            # Calcular escala manteniendo aspect ratio
            img_ancho, img_alto = img.size
            escala = min(area_ancho / img_ancho, area_alto / img_alto)

            nuevo_ancho = img_ancho * escala
            nuevo_alto = img_alto * escala

            # Centrar imagen
            x = (width - nuevo_ancho) / 2
            y = 3*cm

            # Dibujar imagen
            c.drawImage(
                img_reader,
                x, y,
                width=nuevo_ancho,
                height=nuevo_alto,
                preserveAspectRatio=True
            )

        except Exception as e:
            logger.error(f"Error cargando imagen {imagen_path}: {e}")
            c.setFillColor(HexColor('#ef4444'))
            c.setFont('Helvetica', 10)
            c.drawCentredString(width/2, height/2, f"Error cargando imagen: {e}")

        # Pie de página
        c.setFont('Helvetica', 8)
        c.setFillColor(self.COLOR_SECUNDARIO)
        c.drawCentredString(width/2, 1*cm, f"Página {numero + 1}/{total + 2}")

        c.showPage()

    def _generar_pagina_resumen(self, c: canvas.Canvas, config: ConfiguracionModelo):
        """
        Genera una página con grid de miniaturas.

        Args:
            c: Canvas de ReportLab
            config: Configuración del modelo
        """
        width, height = landscape(A4)  # Usar orientación horizontal para el grid
        c.setPageSize(landscape(A4))

        # Encabezado
        c.setFillColor(self.COLOR_PRIMARIO)
        c.setFont('Helvetica-Bold', 14)
        c.drawCentredString(width/2, height - 1.5*cm, "Resumen Visual - Todas las Fotografías")

        # Configurar grid (4 columnas x 5 filas = 20 imágenes)
        cols = 4
        rows = 5

        margen = 1.5*cm
        espacio_x = 0.3*cm
        espacio_y = 0.3*cm

        area_ancho = width - 2*margen
        area_alto = height - 4*cm  # Dejar espacio para encabezado y pie

        thumb_ancho = (area_ancho - (cols - 1) * espacio_x) / cols
        thumb_alto = (area_alto - (rows - 1) * espacio_y) / rows

        # Dibujar miniaturas
        for idx, img_path in enumerate(config.visualizaciones[:20]):  # Máximo 20
            row = idx // cols
            col = idx % cols

            x = margen + col * (thumb_ancho + espacio_x)
            y = height - 3*cm - (row + 1) * (thumb_alto + espacio_y)

            try:
                img = Image.open(img_path)
                img_reader = ImageReader(img)

                # Dibujar miniatura
                c.drawImage(
                    img_reader,
                    x, y,
                    width=thumb_ancho,
                    height=thumb_alto,
                    preserveAspectRatio=True,
                    mask='auto'
                )

                # Etiqueta con nombre de foto
                c.setFont('Helvetica', 6)
                c.setFillColor(self.COLOR_SECUNDARIO)
                foto_id = img_path.stem.split('_')[0] + '_' + img_path.stem.split('_')[1]
                c.drawCentredString(x + thumb_ancho/2, y - 0.3*cm, foto_id)

            except Exception as e:
                logger.error(f"Error en miniatura {img_path}: {e}")

        # Pie de página
        c.setFont('Helvetica', 8)
        c.setFillColor(self.COLOR_SECUNDARIO)
        c.drawCentredString(width/2, 1*cm, f"Página {len(config.visualizaciones) + 2}/{len(config.visualizaciones) + 2}")

        c.showPage()

In [ ]:
def main():
    """Función principal."""
    parser = argparse.ArgumentParser(
        description='Genera PDFs consolidados con visualizaciones de segmentación',
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Ejemplos de uso:
  %(prog)s --directorio-base /resultados
  %(prog)s --directorio-base /resultados --directorio-salida /pdfs_consolidados
  %(prog)s --directorio-base /resultados --modelo mask2former
        """
    )

    parser.add_argument(
        '--directorio-base',
        type=Path,
        required=True,
        help='Directorio base con estructura de modelos'
    )

    parser.add_argument(
        '--directorio-salida',
        type=Path,
        default=None,
        help='Directorio de salida para PDFs (default: directorio-base/pdfs_consolidados)'
    )

    parser.add_argument(
        '--modelo',
        type=str,
        default=None,
        help='Filtrar por modelo específico (ej: mask2former, sam, yolo)'
    )

    args = parser.parse_args()

    # Configurar directorio de salida
    if args.directorio_salida is None:
        args.directorio_salida = args.directorio_base / 'pdfs_consolidados'

    logger.info("=" * 80)
    logger.info("GENERADOR DE PDFs CONSOLIDADOS - VISUALIZACIONES DE SEGMENTACIÓN")
    logger.info("=" * 80)
    logger.info(f"Directorio base: {args.directorio_base}")
    logger.info(f"Directorio salida: {args.directorio_salida}")

    # Escanear configuraciones
    logger.info("\n--- Escaneando configuraciones ---")
    escaneador = EscaneadorVisualizaciones(args.directorio_base)
    configuraciones = escaneador.escanear()

    # Filtrar por modelo si se especifica
    if args.modelo:
        configuraciones = [c for c in configuraciones if args.modelo.lower() in c.modelo_base.lower()]
        logger.info(f"Filtrando por modelo: {args.modelo}")

    if not configuraciones:
        logger.warning("No se encontraron configuraciones")
        return

    logger.info(f"\nTotal configuraciones encontradas: {len(configuraciones)}")

    # Generar PDFs
    logger.info("\n--- Generando PDFs ---")
    generador = GeneradorPDF(args.directorio_salida)

    pdfs_generados = []
    for config in configuraciones:
        try:
            pdf_path = generador.generar_pdf(config)
            pdfs_generados.append(pdf_path)
        except Exception as e:
            logger.error(f"Error generando PDF para {config.codigo_config}: {e}", exc_info=True)

    # Resumen final
    logger.info("\n" + "=" * 80)
    logger.info("RESUMEN")
    logger.info("=" * 80)
    logger.info(f"PDFs generados: {len(pdfs_generados)}/{len(configuraciones)}")
    logger.info(f"Directorio de salida: {args.directorio_salida}")

    if pdfs_generados:
        logger.info("\nPDFs creados:")
        for pdf in sorted(pdfs_generados):
            tamaño_mb = pdf.stat().st_size / (1024 * 1024)
            logger.info(f"  - {pdf.name} ({tamaño_mb:.2f} MB)")


In [ ]:
if __name__ == '__main__':
    main()